In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
from os.path import join
import sys
import json

In [2]:
# Load the "autoreload" extension
%load_ext autoreload

# always reload modules marked with "%aimport"
%autoreload 1

In [3]:
# add the 'src' directory as one where we can import modules
src_dir = join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)

In [4]:
%aimport Data.make_data
from Data.make_data import states_in_nerc
%aimport Analysis.index
from Analysis.index import facility_emission_gen, group_facility_data, facility_co2, adjust_epa_emissions, group_fuel_cats
%aimport util.utils
from util.utils import rename_cols

## Load data

In [116]:
cwd = os.getcwd()
path = join(cwd, '..', 'Data storage',
            'Facility gen fuels and CO2 2017-08-31.zip')
eia_fac = pd.read_csv(path)

In [143]:
eia_fac.head()

,f,fuel,month,plant id,total fuel (mmbtu),year,generation (mwh),elec fuel (mmbtu),geography,last_updated,lat,lon,prime mover,datetime,quarter,all fuel fossil co2 (kg),elec fuel fossil co2 (kg),all fuel total co2 (kg),elec fuel total co2 (kg),type
0,M,SUB,6,10360,0.0,2017,0.0,0.0,USA-WI,2017-08-24T11:46:12-04:00,44.4936,-88.0303,ALL,2017-06-01,2,0.0,0.0,0.0,0.0,COW
1,M,SUB,5,10360,0.0,2017,0.0,0.0,USA-WI,2017-08-24T11:46:12-04:00,44.4936,-88.0303,ALL,2017-05-01,2,0.0,0.0,0.0,0.0,COW
2,M,SUB,4,10360,0.0,2017,0.0,0.0,USA-WI,2017-08-24T11:46:12-04:00,44.4936,-88.0303,ALL,2017-04-01,2,0.0,0.0,0.0,0.0,COW
3,M,SUB,3,10360,0.0,2017,0.0,0.0,USA-WI,2017-08-24T11:46:12-04:00,44.4936,-88.0303,ALL,2017-03-01,1,0.0,0.0,0.0,0.0,COW
4,M,SUB,2,10360,0.0,2017,0.0,0.0,USA-WI,2017-08-24T11:46:12-04:00,44.4936,-88.0303,ALL,2017-02-01,1,0.0,0.0,0.0,0.0,COW


In [118]:
path = join(cwd, '..', 'Data storage', 'Derived data',
            'Monthly EPA emissions 2017-08-31.csv')
epa = pd.read_csv(path)

## Test facility_emissions_gen

In [119]:
fuel_cat_folder = join(cwd, '..', 'Data storage', 'Fuel categories')
state_cats_path = join(fuel_cat_folder, 'State_facility.json')

with open(state_cats_path, 'r') as f:
    state_fuel_cat = json.load(f)
    
custom_cats_path = join(fuel_cat_folder, 'Custom_results.json')
with open(custom_cats_path, 'r') as f:
    custom_fuel_cat = json.load(f)

In [32]:
co2, gen_fuels_custom = facility_emission_gen(eia_facility=eia_fac, epa=epa,
                                              state_fuel_cat=state_fuel_cat,
                                              custom_fuel_cat=custom_fuel_cat,
                                              export_state_cats=False)

Renaming columns
Grouping facilities
Adjusting EPA emissions
Caculating CO2
Gen/fuels to state categories


In [120]:
co2, gen_fuels_state = facility_emission_gen(eia_facility=eia_fac, epa=epa,
                                              state_fuel_cat=state_fuel_cat,
                                              custom_fuel_cat=custom_fuel_cat,
                                              export_state_cats=True)

Renaming columns
Grouping facilities
Adjusting EPA emissions
Caculating CO2
Gen/fuels to state categories


In [9]:
co2.head()

,year,month,plant id,final co2 (kg)
0,2001,1,2,0.000000e+00
1,2001,1,3,9.625411e+08
2,2001,1,4,0.000000e+00
3,2001,1,7,6.700092e+07
4,2001,1,8,7.530204e+08


In [10]:
gen_fuels.head()

,plant id,fuel category,year,month,total fuel (mmbtu),generation (mwh),elec fuel (mmbtu),all fuel fossil co2 (kg),elec fuel fossil co2 (kg),all fuel total co2 (kg),elec fuel total co2 (kg)
0,2,Hydro,2001,1,195479.69,18918.0,195479.69,0.0,0.0,0.0,0.0
1,2,Hydro,2001,2,312128.93,30207.0,312128.93,0.0,0.0,0.0,0.0
2,2,Hydro,2001,3,390060.42,37749.0,390060.42,0.0,0.0,0.0,0.0
3,2,Hydro,2001,4,281925.57,27284.0,281925.57,0.0,0.0,0.0,0.0
4,2,Hydro,2001,5,92594.01,8961.0,92594.01,0.0,0.0,0.0,0.0


### Check generation and fuel consumption totals

Interesting - there is some small part of generation that I'm losing along the way. It's 5 orders of magnitude smaller than the total though.

In [121]:
eia_fac['generation (mwh)'].sum()

65594379743.76258

In [123]:
gen_fuels_state['generation (mwh)'].sum()

65594327524.76352

Fuel consumption is pretty identical though

In [124]:
eia_fac['total fuel (mmbtu)'].sum(), eia_fac['elec fuel (mmbtu)'].sum()

(686393145435.8103, 645653921528.091)

In [125]:
gen_fuels_state['total fuel (mmbtu)'].sum(), gen_fuels_state['elec fuel (mmbtu)'].sum()

(686393145435.7861, 645653921528.0757)

## Extra gen/fuels from non-reporting

In [142]:
%aimport Analysis.index
from Analysis.index import extra_emissions_gen

Total EIA generation/fuel consumption and emission factors

In [128]:
cwd = os.getcwd()
path = join(cwd, '..', 'Data storage',
            'EIA country-wide gen fuel CO2 2017-08-31.csv')
eia_total = pd.read_csv(path)

path = join(cwd, '..', 'Data storage',
            'Final emission factors.csv')
ef = pd.read_csv(path, index_col=0)

In [136]:
extra_co2, extra_gen_fuel = extra_emissions_gen(gen_fuels_state, eia_total, ef)

Results match what I have previously found in the notebooks up on GitHub (Emissions Index repo)

In [94]:
extra_gen_fuel.head()

total fuel (mmbtu)  generation (mwh)  elec fuel (mmbtu)
type year month                                                         
COW  2001 1              16918390.0       1011665.061         13157576.0
          2              14968346.0        911346.865         11574491.0
          3              18321543.0       1153328.913         14765162.0
          4              13530117.0        776341.623         10374008.0
          5              14014024.0        801266.923         10920463.0

In [95]:
eia_extra.loc[idx['DPV',:,:]]

total fuel (mmbtu)  generation (mwh)  elec fuel (mmbtu)
type year month                                                         
DPV  2014 1                     NaN         624032.82                NaN
          2                     NaN         663828.73                NaN
          3                     NaN         907029.95                NaN
          4                     NaN         988423.77                NaN
          5                     NaN        1092011.80                NaN
          6                     NaN        1100560.82                NaN
          7                     NaN        1148527.79                NaN
          8                     NaN        1139134.66                NaN
          9                     NaN        1046360.70                NaN
          10                    NaN         964877.96                NaN
          11                    NaN         791801.55                NaN
          12                    NaN         766068.15                NaN
     2015 1                     NaN         746445.84                NaN
          2                     NaN         815764.50                NaN
          3                     NaN        1133577.23                NaN
          4                     NaN        1263528.30                NaN
          5                     NaN        1393836.75                NaN
          6                     NaN        1408013.52                NaN
          7                     NaN        1486681.59                NaN
          8                     NaN        1468048.33                NaN
          9                     NaN        1329651.46                NaN
          10                    NaN        1197707.54                NaN
          11                    NaN         982048.27                NaN
          12                    NaN         913963.95                NaN
     2016 1                     NaN        1020588.78                NaN
          2                     NaN        1189469.53                NaN
          3                     NaN        1582361.09                NaN
          4                     NaN        1763404.74                NaN
          5                     NaN        1944571.47                NaN
          6                     NaN        1991456.73                NaN
          7                     NaN        2065890.59                NaN
          8                     NaN        2003636.20                NaN
          9                     NaN        1789836.27                NaN
          10                    NaN        1607185.85                NaN
          11                    NaN        1306748.99                NaN
          12                    NaN        1201677.95                NaN
     2017 1                     NaN        1282339.18                NaN
          2                     NaN        1449156.85                NaN
          3                     NaN        2054274.30                NaN
          4                     NaN        2267811.28                NaN
          5                     NaN        2479480.14                NaN
          6                     NaN        2524250.11                NaN

## Total CO₂ (national)

In [130]:
facility_co2 = co2.groupby(['year', 'month']).sum()

In [131]:
facility_co2.tail()

plant id  final co2 (kg)
year month                          
2017 2      57482310    1.165442e+11
     3      58485598    1.237187e+11
     4      58955690    1.133454e+11
     5      59223601    1.292581e+11
     6      59451819    1.512113e+11

In [141]:
extra_co2.loc[idx['NG', :, :],:].tail(n=11)

all fuel co2 (kg)  elec fuel co2 (kg)
type year month                                       
NG   2016 8           6.022574e+09        4.039321e+09
          9           4.917984e+09        3.118869e+09
          10          4.312456e+09        2.624005e+09
          11          4.231444e+09        2.458983e+09
          12          4.418713e+09        2.530048e+09
     2017 1           7.823823e+09        6.101011e+09
          2           6.601814e+09        5.030973e+09
          3           8.630216e+09        6.986014e+09
          4           7.823192e+09        6.280070e+09
          5           8.749634e+09        7.244919e+09
          6           1.000377e+10        8.465898e+09

In [133]:
national_co2 = (facility_co2.loc[:, 'final co2 (kg)']
                + extra_co2.loc[:, 'elec fuel co2 (kg)'])

These values are very close, but not quite exactly the same as what I've put up on the website.

Why? Could it be because the extra 

In [134]:
national_co2

year  month
2001  1        2.162121e+11
      2        1.814475e+11
      3        1.900841e+11
      4        1.761317e+11
      5        1.905779e+11
      6        2.062662e+11
      7        2.311815e+11
      8        2.404311e+11
      9        1.950060e+11
      10       1.845210e+11
      11       1.731834e+11
      12       1.882633e+11
2002  1        1.951862e+11
      2        1.709727e+11
      3        1.855136e+11
      4        1.759043e+11
      5        1.863580e+11
      6        2.081023e+11
      7        2.385440e+11
      8        2.355222e+11
      9        2.092338e+11
      10       1.948277e+11
      11       1.844629e+11
      12       2.023788e+11
2003  1        2.167834e+11
      2        1.880347e+11
      3        1.876779e+11
      4        1.727763e+11
      5        1.840170e+11
      6        2.007602e+11
                   ...     
2015  1        1.779027e+11
      2        1.696220e+11
      3        1.522496e+11
      4        1.310557e+11
      5 

## Facility state and lat/lon file generation

In [148]:
%aimport Data.make_data
from Data.make_data import facility_location_data

In [150]:
facility_location_data(eia_fac)